In [2]:
#import required libraries
import geopandas as gpd
import xarray as xr
import pandas as pd
import os, glob
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.colors
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.cm as cm

import rioxarray as rio
import rasterio
from rasterio.crs import CRS
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.windows import Window, transform as rio_window_transform
from rasterio.windows import from_bounds
from rasterio.transform import Affine
from rasterio.features import rasterize
from rasterio.transform import from_origin
from rasterio.features import rasterize
from shapely.geometry import box

import dask
import dask.array as da
from dask.delayed import delayed

from osgeo import gdal, ogr, osr

import seaborn as sns
import zipfile

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
Q:\UserTemp\ubach\AppData\Local\Temp\4\ipykernel_27256\3466570486.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https:

In [3]:
# Path to data folders
project_f = r'L:\f02_data\wildfires\spatial_data'

# Create output folder structure
outfolder =  os.path.join(project_f, 'output')
os.makedirs(outfolder, exist_ok=True)

outFIREID_folder =  os.path.join(project_f, 'output', 'FIRE_ID')
os.makedirs(outFIREID_folder, exist_ok=True)

outFIREYEARDOY_folder =  os.path.join(project_f, 'output', 'FIRE_YEARDOY')
os.makedirs(outFIREYEARDOY_folder, exist_ok=True)

In [4]:
# load wildfires data
effis_path =  os.path.join(project_f, 'input', 'effis_rawdata_20240402', 'modis.ba.poly.shp')
effis = gpd.read_file(effis_path)

# read effis id as an integer
effis['id'] = effis['id'].astype(int)

# Convert the column containing dates to datetime format
effis['date'] = pd.to_datetime(effis['FIREDATE'])

# Change the CRS to EPSG:3035
effis = effis.to_crs(epsg=3035)

# Verify the new CRS
print("New CRS:", effis.crs)

New CRS: EPSG:3035


In [6]:
effis2001 = effis.query('Year==2001')

In [8]:
effis2001.query('id==178883')

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,...,OTHERNATLC,AGRIAREAS,ARTIFSURF,OTHERLC,PERCNA2K,CLASS,Year,Month,geometry,date
3567,178883,2001-12-16 00:00:00,2001-12-17 00:00:00,ES,Ourense,Laza,248,0,0,0,...,73.09236947788229,21.28514056224045,0,0,0,FireSeason,2001,12,"POLYGON ((2889155.230 2278644.139, 2890216.352...",2001-12-16
